# Домашнее задание к лекции «Продвинутый pandas»

## Преподаватель: Константин Башевой
## Домашнее задание

## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [2]:
import pandas as pd
import json

In [3]:
purchases = dict()
i = 0

with open('Data/purchase_log.txt') as f:
    next(f)
    for line in f:
        line = line.strip()
        dict_ = json.loads(line)
        purchases[str(dict_['user_id'])] = str(dict_['category'])        
        #i += 1


In [4]:
log = pd.DataFrame.from_dict(purchases,orient='index',columns=['category'] )
log['user_id'] = log.index
log = log.reset_index()
log = log.drop(columns = ['index'])

In [5]:
log = pd.read_csv('Data/visit_log.csv', delimiter = ';')

In [6]:
log['source_type'] = log['traffic_source'] 
log.loc[((log['traffic_source'] == 'yandex') | (log['traffic_source'] == 'google')), 'source_type'] = 'organic' 
log.loc[(((log['traffic_source'] == 'paid') | (log['traffic_source'] == 'email'))) & (log['region'] != 'Russia'), 'source_type'] = 'other'
log.loc[(((log['traffic_source'] == 'paid') | (log['traffic_source'] == 'email'))) & (log['region'] == 'Russia'), 'source_type'] = 'ad'


In [7]:
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном


In [12]:
URLs = pd.read_csv('Data/URLs.txt')

In [14]:
URLs.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [17]:
URLs[URLs['url'].str.contains('/[0-9]{8}-', regex=True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


## Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [98]:
ratings = pd.read_csv('Data/ml-latest-small/ratings.csv')

In [99]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [100]:
active_users = pd.DataFrame(data = ratings['userId'].value_counts())
active_users_idx = active_users[active_users['userId'] > 100].index

In [101]:
def avg_life(idx):
    ratings_user = ratings.copy()
    ratings_user = ratings_user.loc[ratings_user['userId'] == idx]
    return ratings_user['timestamp'].max() - ratings_user['timestamp'].min()
    

In [102]:
ratings_active = ratings[ratings['userId'].isin(active_users_idx)]
ratings_active['Life'] = ratings_active['userId'].apply(avg_life)
ratings_active = ratings_active[{'userId','Life'}]
ratings_active = ratings_active.drop_duplicates()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [103]:
ratings_active.head(5)

,userId,Life
147,4,203560
583,8,85187
962,15,471393496
2691,17,8053
3105,19,5282


In [117]:
print('Average life: ',ratings_active['Life'].mean())

Average life:  40080507.4496124


## Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [85]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [86]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [87]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [88]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [118]:
# получение таблицы по всем типам платежей без адресов: 
result0 = pd.merge(air, auto, how='outer', on='client_id', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

result = pd.merge(result0, rzd, how='outer', on='client_id', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [119]:
result

,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,NaN,NaN,1093.0
1,112,NaN,NaN,2810.0
2,113,NaN,57483.0,10283.0
3,114,NaN,83.0,5774.0
4,115,81.0,912.0,981.0
5,116,4.0,4834.0,NaN
6,117,13.0,98.0,NaN
7,118,173.0,NaN,NaN


In [120]:
# получение таблицы по всем типам платежей с адресами: 
result_adress = pd.merge(result, client_base, how='outer', on='client_id', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [121]:
result_adress

,client_id,air_revenue,auto_revenue,rzd_revenue,address
0,111,NaN,NaN,1093.0,Комсомольская 4
1,112,NaN,NaN,2810.0,Энтузиастов 8а
2,113,NaN,57483.0,10283.0,Левобережная 1а
3,114,NaN,83.0,5774.0,Мира 14
4,115,81.0,912.0,981.0,ЗЖБИиДК 1
5,116,4.0,4834.0,NaN,Строителей 18
6,117,13.0,98.0,NaN,Панфиловская 33
7,118,173.0,NaN,NaN,Мастеркова 4
